In [1]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud


In [2]:
job_data = pd.read_csv('./data/SF_jobs.csv')
job_data.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
#Functions for data cleaning
wnl = WordNetLemmatizer()

def text_cleaner(text):
    
    text = text.lower().split()
    text = [wnl.lemmatize(myWord) for myWord in text] # Lemmertization

    stop_words = set(stopwords.words("english")) # Filter out any stop words
    text = [w for w in text if not w in stop_words]
    
    return text # return list of tokens

In [4]:
cleanDescription = []
for i in range(job_data.shape[0]):
    cleanDescription.append(text_cleaner(job_data.job_description[i]))

In [7]:
len(cleanDescription)

1547

In [10]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=20,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [11]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [12]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
16     14.735337        1       1 -0.133274  0.165279
14     14.002904        1       2 -0.096625 -0.080790
3      12.894173        1       3 -0.118230 -0.127374
1       9.472486        1       4 -0.106205  0.015314
9       7.805078        1       5 -0.093907 -0.047348
0       7.485810        1       6 -0.121730  0.220279
10      6.062873        1       7 -0.079512 -0.053608
15      5.924860        1       8 -0.085590 -0.092750
2       4.677366        1       9 -0.099964  0.122072
6       2.952126        1      10 -0.013750 -0.088172
8       2.921219        1      11 -0.040865 -0.042587
13      2.645771        1      12 -0.014725  0.001992
18      2.331410        1      13  0.018243  0.017911
12      2.318241        1      14 -0.004474 -0.020267
4       1.924838        1      15  0.038010 -0.109192
5       0.760246        1      16  0.158122  0.014117
19      0.582791        1      17  0.147276  0.017549
7       0.317610        1      18  0.216804  0.080567
11      0.174374        1      19  0.202078 -0.003191
17      0.010486        1      20  0.228320  0.010200, topic_info=     Category         Freq                                      Term  \
term                                                                   
613   Default  2290.000000                                      data   
1034  Default  1167.000000                                experience   
2891  Default   584.000000                                  business   
319   Default   266.000000                                         .   
3137  Default   140.000000                                 reporting   
6263  Default   114.000000                                    report   
2626  Default   403.000000                                      year   
2055  Default   658.000000                                      work   
7735  Default   199.000000                               requirement   
805   Default   126.000000                                     spark   
4173  Default   257.000000                                    resume   
1328  Default   298.000000                                 technical   
7094  Default   150.000000                                 scientist   
6204  Default   211.000000                                      tool   
359   Default   274.000000                                       big   
854   Default   363.000000                                technology   
7337  Default   228.000000                                  analysis   
2779  Default   204.000000                                  learning   
208   Default   239.000000                                   project   
1680  Default   402.000000                                    system   
7533  Default   240.000000                                     apply   
2853  Default    78.000000                                      real   
7743  Default   310.000000                                   ability   
3873  Default   443.000000                                     skill   
5435  Default   134.000000                                   forward   
2011  Default   251.000000                                         +   
105   Default   168.000000                                   machine   
447   Default   247.000000                                   support   
4692  Default   139.000000                                 protected   
3533  Default   128.000000                                     click   
...       ...          ...                                       ...   
46    Topic20     0.000989                                drivenroot   
4272  Topic20     0.000989                                    withor   
4018  Topic20     0.000989  schedulerisaplus.experienceinhealthcarea   
4966  Topic20     0.026791                                     field   
3451  Topic20     0.008394                                   similar   
237   Topic20     0.006627       